In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from src.component.data_info import *
from datetime import datetime, timedelta

In [2]:
# specify the date we want to fetch our data
from datetime import datetime, timedelta

import pandas as pd
#specify the current time
current_date = pd.to_datetime(datetime.utcnow(),utc=True).floor('h')
print(f'{current_date=}')

# we fetch raw data for the last 28 days, to add redundancy to our data pipeline
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=90)#it should only fetch for the last hour but because we dont want the pipline to break in case this data or the provois our was not availble so thats why we added for the entier month
fetch_data_from = pd.Timestamp(year=fetch_data_from.year, month=fetch_data_from.month, day=1,tz='UTC')
print(fetch_data_to, fetch_data_from )

current_date=Timestamp('2025-04-05 12:00:00+0000', tz='UTC')
2025-04-05 12:00:00+00:00 2025-01-01 00:00:00+00:00


In [3]:
#fetch batch of data from the wherehous

def fetch_batch_raw_data(from_date: datetime, to_date: datetime) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e. 1 year)
    """
    from_date_ = from_date - timedelta(days=7*52)
    to_date_ = to_date - timedelta(days=7*52)
    print(f'{from_date_=}, {to_date_=}')

    # download 2 files from website
    demand_values = load_full_data(from_date_,to_date_)
    demand_values = demand_values[(demand_values.date >= from_date_) & (demand_values.date <= to_date_)]


    # shift the data to pretend this is recent data
    demand_values['date'] += timedelta(days=7*52)

    demand_values.sort_values(by=['sub_region_code', 'date'], inplace=True)

    return demand_values

In [4]:
data=fetch_batch_raw_data(fetch_data_from,fetch_data_to)

from_date_=Timestamp('2024-01-03 00:00:00+0000', tz='UTC'), to_date_=Timestamp('2024-04-06 12:00:00+0000', tz='UTC')
Loading file D:\projects\electricity_demand_predictor-main\data\raw\electricity_raw_data\hourly_demand_2024-01-03.json
Loading file D:\projects\electricity_demand_predictor-main\data\raw\electricity_raw_data\hourly_demand_2024-01-04.json
Loading file D:\projects\electricity_demand_predictor-main\data\raw\electricity_raw_data\hourly_demand_2024-01-05.json
Loading file D:\projects\electricity_demand_predictor-main\data\raw\electricity_raw_data\hourly_demand_2024-01-06.json
Loading file D:\projects\electricity_demand_predictor-main\data\raw\electricity_raw_data\hourly_demand_2024-01-07.json
Loading file D:\projects\electricity_demand_predictor-main\data\raw\electricity_raw_data\hourly_demand_2024-01-08.json
Loading file D:\projects\electricity_demand_predictor-main\data\raw\electricity_raw_data\hourly_demand_2024-01-09.json
Loading file D:\projects\electricity_demand_predic

In [5]:
# string to datetime
data['date'] = pd.to_datetime(data['date'], utc=True)

# add column with Unix epoch milliseconds
data['seconds'] = data['date'].astype('int64') // 10**6

In [6]:
data.isna().sum()

date               0
sub_region_code    0
demand             0
temperature_2m     0
seconds            0
dtype: int64

In [7]:
data.tail()

,date,sub_region_code,demand,temperature_2m,seconds
25992,2025-04-05 08:00:00+00:00,10,1669,15.408501,1743840000000
25981,2025-04-05 09:00:00+00:00,10,1675,16.608500,1743843600000
25970,2025-04-05 10:00:00+00:00,10,1718,18.958500,1743847200000
25959,2025-04-05 11:00:00+00:00,10,1789,20.008501,1743850800000
25948,2025-04-05 12:00:00+00:00,10,1850,20.808500,1743854400000


In [8]:
print("Min date:", data["date"].min())
print("Max date:", data["date"].max())


Min date: 2025-01-01 04:00:00+00:00
Max date: 2025-04-05 12:00:00+00:00


In [9]:
# from src.component.data_info import *
# import tqdm
# features, targets = transform_ts_data_into_features_and_target(
#     data,
#     input_seq_len=24*28*1, # one month
#     step_size=23,
# )

In [10]:
# features

In [11]:
# targets

In [12]:
import hopsworks
from src.component import feature_group_config

In [13]:
hopsworks.__version__

'4.1.8'

In [14]:
# project = hopsworks.login(
#     project=feature_group_config.HOPSWORKS_PROJECT_NAME,
#     api_key_value=feature_group_config.HOPSWORKS_API_KEY
# )
# feature_store = project.get_feature_store()

# FEATURE_GROUP_NAME = 'electricity_demand_feature_group2'
# FEATURE_GROUP_VERSION = 3

# feature_group = feature_store.get_or_create_feature_group(
#     name=FEATURE_GROUP_NAME,
#     version=FEATURE_GROUP_VERSION,
#     description="Time-series data at hourly frequency",
#     primary_key = ['sub_region_code', 'date'],
#     event_time='date',
# )

In [15]:
#save predictions to the feature store
from src.component.feature_store_api import get_feature_store
import src.component.feature_group_config as config

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=1,
    description="Time-series data at hourly frequency",
    primary_key = ['sub_region_code', 'date'],
    event_time='date',
)



HOPSWORKS_PROJECT_NAME:  energy_demand_prediction
2025-04-05 18:16:45,850 INFO: Initializing external client
2025-04-05 18:16:45,851 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-04-05 18:16:47,149 WARNING: UserWarning: The installed hopsworks client version 4.1.8 may not be compatible with the connected Hopsworks backend version 4.2.0. 
To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'



2025-04-05 18:16:48,261 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213640


In [16]:
feature_group.insert(data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |██████████| Rows 25938/25938 | Elapsed Time: 00:13 | Remaining Time: 00:00


Launching job: electricity_demand_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1213640/jobs/named/electricity_demand_feature_group_1_offline_fg_materialization/executions


(Job('electricity_demand_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)